In [1]:
# https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git

In [2]:
#git remote add origin https://github.com/johanna23cct/integrated-CA2-MSc-2023094.git
#git branch -M main
#git push -u origin main

In [3]:
pip install Matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bokeh

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install skforecast --user

Note: you may need to restart the kernel to use updated packages.


In [75]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import nltk
#nltk.dowload()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import download

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
#from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

# Spark
# import SparkSession library 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, concat_ws, regexp_replace, split, size, length, broadcast
from pyspark.sql.types import DoubleType, StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.types import *
import pyspark.sql.types as typ
import pyspark.sql.functions as fn
import pyspark.sql.functions as F


# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

In [76]:
# SparkContext HOW I CAN SHOW THAT ... SC  It is not working to me, in the first time!
# that ok now, I know, is sc.master.  I was put only sc
sc.master

'local[*]'

In [77]:

spark = SparkSession.builder.appName("data_projectTweets").getOrCreate()

In [78]:
# creating a customer dataframe by declaring the schema and passing values


In [79]:
schema = StructType().add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [80]:
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
#                                   Start whit the Data
#==========================================================================================

In [81]:
# Big headache and colapse, to read the dataset
# I was load the file from my hadoop (local)
# df = spark.read.csv('home/hduser/Documnets/CA2', header+True, inferSchema=True)

In [82]:
#folowiong you can see the two way to load the dataset, 
#    (command)+(file://)+(/path/)+(filename)
#I will use direct from the path, to have one of five V' (Velocity) 

path = "/user1/ProjectTweets.csv"
#df = spark.read.csv("file:///home/hduser/Documents/CA2/ProjectTweets.csv", header=True, inferSchema = True)

In [83]:
#reading the data. i can't belive, it is working

In [84]:
tweets = sc.textFile("file:///home/hduser/Documents/CA2/ProjectTweets.csv")
header = tweets.first()

tweets = tweets \
    .filter(lambda row: row != header) \
    .map(lambda row: [int(elem) for elem in row.split(',')])

In [85]:
# creating the schema for my DataFrame
# was one  error because i didn' import pyspark.sql.types as typ
# but next did it, the command worked, great!

In [86]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True)
        for h in header.split(',')
    ]
]
schema = typ.StructType(fields)

In [87]:
column_names  = ['_c0','Ids', 'Date', 'Flag', 'User', 'Text']    
full_df = pd.read_csv('file:///home/hduser/Documents/CA2/ProjectTweets.csv', header=None, names=column_names)

In [88]:
full_df.head()

,_c0,Ids,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [89]:
# display the total number of rows data
total_rows = full_df.shape[0]
formatted_total_rows = "{:,}".format(total_rows)
print("Total Rows:", formatted_total_rows)

Total Rows: 1,600,000


In [90]:
# rename the file whit  row's title

In [91]:
full_df.to_csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv")

In [92]:
Header_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [93]:
# Drop unnecessary column

In [94]:
#df = full_df.drop(['Flag', 'Unnamed: 0', ], axis=NO_QUERY)
#df.dropna(inplace=True)

Header_df.drop('Flag').show()

+---+----------+--------------------+---------------+--------------------+
|_c0|       Ids|                Date|           User|                Text|
+---+----------+--------------------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|2Hood4Hollywood|@Tatiana_K nope t...|
|  9|1467812025|Mon Apr 06 22:20:...|        mimismo|@twittera que me ...|
| 10|1467812416|Mon Apr 0

2023-11-03 02:44:44,839 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, User, Text
 Schema: _c0, Ids, Date, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


In [95]:
schema = StructType().add("_c0","integer").add("Ids","integer").add("Date","string").add("Flag", "string").add("User",'string').add("Text", "string")

In [96]:
full_df.to_csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv")

In [97]:
# Creating a UDF to apply VADER sentiment analysis to a Text column

In [98]:
def analyze_sentiment(Text):
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(Text)
    return sentiment['compound']

In [99]:
#register the UDF

In [100]:
sentiment_udf = udf(analyze_sentiment, DoubleType())

In [101]:
#Apply sentiment

In [102]:
Header_df = Header_df.withColumn("sentiment", sentiment_udf(Header_df["Text"]))

In [103]:
print(Header_df.show())

2023-11-03 02:44:55,830 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Ids, Date, Flag, User, Text
 Schema: _c0, Ids, Date, Flag, User, Text
Expected: _c0 but found: 
CSV file: file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv


+---+----------+--------------------+--------+---------------+--------------------+---------+
|_c0|       Ids|                Date|    Flag|           User|                Text|sentiment|
+---+----------+--------------------+--------+---------------+--------------------+---------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|  -0.0173|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|    -0.75|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|   0.4939|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|    -0.25|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|  -0.6597|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|      0.0|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |   0.4767|
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZ

In [104]:
#why i have Flag  again??
print(column_names[:5])

['_c0', 'Ids', 'Date', 'Flag', 'User']


In [105]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>

In [106]:
# Creating my Dataframe (i did it in line 27 )

In [107]:
#        Basic Feature Extraction

In [108]:
Header_df = spark.read.csv("file:///home/hduser/Documents/CA2/Header_Proj_Tweets.csv", header=True, inferSchema = True)

In [109]:
Tweets = Header_df

In [110]:
Tweets = Tweets.withColumn("word_count", size(split(Tweets["Text"], " ")))

In [111]:
# Number of Words  

In [112]:
#Tutorial did't work because is a Panda code: TypeError: 'Column' object is not callable

#Tweets['word_count'] = Tweets['Text'].apply(lambda x: len(str(x).split("")))
#Tweets[['Text','word_count']].head()

In [113]:
Tweets.select("Text", "word_count").show()

+--------------------+----------+
|                Text|word_count|
+--------------------+----------+
|@switchfoot http:...|        20|
|is upset that he ...|        22|
|@Kenichan I dived...|        19|
|my whole body fee...|        11|
|@nationwideclass ...|        22|
|@Kwesidei not the...|         6|
|         Need a hug |         4|
|@LOLTrish hey  lo...|        24|
|@Tatiana_K nope t...|         7|
|@twittera que me ...|         6|
|spring break in p...|         8|
|I just re-pierced...|         6|
|@caregiving I cou...|        21|
|@octolinz16 It it...|        16|
|@smarrison i woul...|        23|
|@iamjazzyfizzle I...|        20|
|Hollis' death sce...|        19|
|about to file taxes |         5|
|@LettyA ahh ive a...|        12|
|@FakerPattyPattz ...|        13|
+--------------------+----------+
only showing top 20 rows



In [114]:
# Number of CCharaccters

In [115]:
#Tutorial did't work because is a Panda code:TypeError: 'Column' object is not callable
#Tweets['char_count'] = Tweets['Text'].str.len()
## this also includes spaces
#Tweets[['Text','char_count']].head()

In [116]:
Tweets = Tweets.withColumn("char_count", length(Tweets["Text"]))

In [117]:
Tweets.select("Text", "char_count").show()

+--------------------+----------+
|                Text|char_count|
+--------------------+----------+
|@switchfoot http:...|       115|
|is upset that he ...|       111|
|@Kenichan I dived...|        89|
|my whole body fee...|        47|
|@nationwideclass ...|       111|
|@Kwesidei not the...|        29|
|         Need a hug |        11|
|@LOLTrish hey  lo...|        99|
|@Tatiana_K nope t...|        36|
|@twittera que me ...|        25|
|spring break in p...|        43|
|I just re-pierced...|        26|
|@caregiving I cou...|        94|
|@octolinz16 It it...|        77|
|@smarrison i woul...|       117|
|@iamjazzyfizzle I...|       103|
|Hollis' death sce...|        93|
|about to file taxes |        20|
|@LettyA ahh ive a...|        64|
|@FakerPattyPattz ...|        79|
+--------------------+----------+
only showing top 20 rows



In [118]:
#Basic Pre-processing

In [119]:
Tweets = Tweets.withColumn("Text", concat_ws("", lower(col("Text"))))

In [120]:
#Removing Punctuation 

In [121]:
Tweets = Tweets.withColumn("Text", regexp_replace(col("Text"), r'[^\w\s]', ''))

In [122]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|is upset that he ...|
|kenichan i dived ...|
|my whole body fee...|
|nationwideclass n...|
|kwesidei not the ...|
|         need a hug |
|loltrish hey  lon...|
|tatiana_k nope th...|
|twittera que me m...|
|spring break in p...|
|i just repierced ...|
|caregiving i coul...|
|octolinz16 it it ...|
|smarrison i would...|
|iamjazzyfizzle i ...|
|hollis death scen...|
|about to file taxes |
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [123]:
# Removal of Stop Words
#Load stop word once and broadcast it to all worker nodes

In [124]:

stop  = stopwords.words('english')
broadcasted_stopwords = spark.sparkContext.broadcast(stop)

In [125]:
#Define a funcion to remove stopwords

In [126]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop]
    return " ".join(filtered_words)

In [127]:
#Register the UDF

In [128]:
remove_stopwords_udf = udf(remove_stopwords, StringType())

In [129]:
#Apply the UDF

In [130]:
Tweets = Tweets.withColumn("Text", remove_stopwords_udf(Tweets["Text"]))

In [131]:
Tweets.select("Text").show()

+--------------------+
|                Text|
+--------------------+
|switchfoot httptw...|
|upset cant update...|
|kenichan dived ma...|
|whole body feels ...|
|nationwideclass b...|
| kwesidei whole crew|
|            need hug|
|loltrish hey long...|
|tatiana_k nope didnt|
|  twittera que muera|
|spring break plai...|
|      repierced ears|
|caregiving couldn...|
|octolinz16 counts...|
|smarrison wouldve...|
|iamjazzyfizzle wi...|
|hollis death scen...|
|          file taxes|
|lettya ahh ive al...|
|fakerpattypattz o...|
+--------------------+
only showing top 20 rows



In [132]:
#= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
#=                          T O K E N I Z A T I O N                                      =
#= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

In [ ]:
#Data Procesising in panda

In [ ]:
#df = df.withColumn("Text", lower(df["text"]))
#df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
df = df.withColumn("Text", lower(df["text"]))
df = df.withColumn("Text", regexp_replace(df["text"], "[^a-z0-9\\s]", ""))

In [ ]:
rows_df.shape